In [129]:
import json, gzip,ujson
import pandas as pd 
import numpy as np, toml
from typing import Tuple, List, Dict
import mysql.connector as mariadb
import warnings    
warnings.filterwarnings("ignore")


import sys
sys.path.insert(0, '/Users/anis/test_labnbook/Indicators/scripts')  

from flow_3 import * 
from utils.utils import *

def get_writing_time(id_labdoc : int, trace : pd.DataFrame)-> dict:
    """
    Calculates the number of modification (trace 9) for each change of authors and also multiplies it by 20 or 30 to have the effective writing time 
    output : ex : {445494: [(12673, 15, 450), (12661, 2, 60), (12673, 47)]} 
    """ 
    labdoc  = trace[trace['id_labdoc'] == id_labdoc]

    labdoc_time = labdoc["action_time"].iloc[0]
    date_to_compare = pd.Timestamp('2020-05-16')

    if labdoc_time.date() < date_to_compare.date():
        factor = 20
    else:
        factor = 30
    count_list = []
    current_author = None
    current_count = 0
    for author in labdoc['id_user']:
        if author == current_author:
            current_count += 1
        else:
            if current_author is not None:
                count_list.append((current_author, current_count,current_count*factor))
            current_author = author
            current_count = 1
    count_list.append((current_author, current_count))
    
    return {int(id_labdoc) : count_list}

In [149]:

def get_times(config_db, df_summary_nonsemantic_indicators, df_semantic_indicator):
    """
    description
    """
    df_all_0 = pd.merge(df_summary_nonsemantic_indicators,df_semantic_indicator,"inner")

    try:
        conn = mariadb.connect(user=config_db['user'], password=config_db['password'],
                        host=config_db['host'], database=config_db['database_name'])
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
    pass
    cursor = conn.cursor()
    id_trace = df_all_0["id_trace"]
    trace = pd.read_sql(f" SELECT id_labdoc, id_trace ,action_time from trace WHERE  id_trace in {tuple(id_trace)} Order By  id_trace ASC", conn)
    df_all_0["id_trace"] = df_all_0["id_trace"].astype(np.int64)
    df_all_0["id_labdoc"] = df_all_0["id_labdoc"].astype(np.int64)
    # print(trace.info(),'\n',df_all_0.info())
    df_all = pd.merge(trace, df_all_0, 'right')
    df_all.to_csv("data/tmp/reports/summary_all.csv")
    #-----

    # # Filter le nombre de labdoc disponible 
    id_labdoc = df_all_0["id_labdoc"]
    trace = pd.read_sql( f" SELECT id_labdoc,id_user, action_time from trace WHERE id_labdoc in {tuple(id_labdoc)}  AND id_action=9 Order By id_labdoc ASC, action_time ASC", conn)

    res = []
    for selectec_labdoc in id_labdoc:
        res.append(get_writing_time(int(selectec_labdoc),trace))
    
    # with open("data/tmp/reports/times.json", "w") as f :
    #     json.dump(res,f)



In [150]:
with open("/Users/anis/test_labnbook/Indicators/pyproject.toml", "r") as f:
    config = toml.loads(f.read())

df_semantic_indicator = semantic_indicator_csv("/Users/anis/test_labnbook/Indicators/data/tmp/reports/semantic.json")
df_summary_nonsemantic_indicators = summary_nonsemantic_indicators_csv("/Users/anis/test_labnbook/Indicators/data/tmp/2_collab.json.gz")
get_times(config["database"], df_summary_nonsemantic_indicators, df_semantic_indicator)


OSError: Cannot save file into a non-existent directory: 'data/tmp/reports'

In [141]:
trace.loc[0]

id_labdoc                   445494
id_user                      12673
action_time    2021-03-16 09:59:11
Name: 107179, dtype: object

In [145]:
x = trace[trace["id_labdoc"]  == 223]

In [146]:
x.iloc[0]

IndexError: single positional indexer is out-of-bounds